# Deep dive on differential programming

# Custom differential data types


In [0]:
struct Point: Differentiable {
    var x: Double
    var y: Double

    // Define an arbitrary method on our type, and ask the compiler to compute
    // the derivative at compile time.
    @differentiable
    func foo() -> Double {
        return x * x + 2 * y
    }
}

In [0]:
// Define an arbitrary method taking two different differentiable types, and
// request the compiler compute the derivative at compile time.
@differentiable
func bar(_ point: Point, _ real: Double) -> Double {
  return (point.foo() * real) + (point.y) / real - point.x / 2
}

In [0]:
print(gradient(at: Point(x: 3, y: 1)) { point in bar(point, 7.1) })
print(gradient(at: Point(x: 5, y: -17.3)) { point in bar(point, 0.3) })

In [0]:
let point = Point(x: 0.3, y: -1.7)
print(gradient(at: 1.3) { real in bar(point, real) })
print(gradient(at: -0.4) { real in bar(point, real) })

In [0]:
print(gradient(at: Point(x: 0.3, y: -1.7), 0.6) { point, real in bar(point, real)})

In [0]:
let (val, grad) = valueWithGradient(at: Point(x: -1.7,
                                    y: 0.3), 0.6) { point, real in
  bar(point, real)
}

print("The value of bar is: \(val)")

// Grad is a tuple, containing the partial derivatives for the complex number,
// and the real number.
print("The gradients of bar are: \(grad.0), and \(grad.1)")

# Fibonacci Activations


In [0]:
// A recursive implementation of the Fibonacci function.
func fib(_ n: Int) -> Int {
  if n <= 2 {
    return 1
  }
  return fib(n - 1) + fib(n - 2)
}

In [0]:
// We define a typealias for clarity (and to help type inference).
typealias ActivationFunction = @differentiable (Float32) -> Float32

In [0]:
import TensorFlow // For round function

let fibActivation: ActivationFunction = differentiableFunction { input in
  let baseNumber = Int(round(input))  // Round to the nearest integer.
  let prev = fib(baseNumber - 1)
  let doublePrev = fib(baseNumber - 2)
  // Return the value, and a closure to compute the backward pass.
  return (Float32(prev + doublePrev),
          { chain in Float(doublePrev) + chain })
}

In [0]:
// Compute the value and the gradient for our activation function for an input.
valueWithGradient(at: 17) { fibActivation($0)}